**Number of Unique Customers**

In [1]:
SELECT 
COUNT(DISTINCT customer_id) total_customers
FROM subscriptions

(1 row affected)

Total execution time: 00:00:00.258

total_customers
1000


**Monthly Distribution of Trial Plans**

In [1]:
SELECT 
COUNT(plan_id) plan_count, 
MONTH(start_date) month
FROM subscriptions
WHERE plan_id = 0
GROUP BY MONTH(start_date)
ORDER BY month

(12 rows affected)

Total execution time: 00:00:00.038

plan_count,month
88,1
68,2
94,3
81,4
88,5
79,6
89,7
88,8
87,9
79,10


**Customers per Plan 2020**

In [2]:
;WITH cte AS (
SELECT
plan_name, 
s.plan_id, 
start_date
FROM subscriptions s
JOIN plans p
	ON s.plan_id = p.plan_id
WHERE YEAR(start_date) > '2020'
)
SELECT COUNT(plan_name) AS plan_count_2021, plan_name, plan_id
FROM cte
WHERE plan_id in (0, 1, 2, 3, 4, 5)
GROUP BY 
plan_name,
plan_id
ORDER BY plan_id

(4 rows affected)

Total execution time: 00:00:00.062

plan_count_2021,plan_name,plan_id
8,basic monthly,1
60,pro monthly,2
63,pro annual,3
71,churn,4


**Number & Percentage of Customers who Churned**

In [6]:
SELECT 
COUNT(*) AS churn_count, 
CAST(100.0 * COUNT(*) / (SELECT COUNT(DISTINCT customer_id) FROM subscriptions) AS float) AS '%'
FROM subscriptions
WHERE plan_id = 4

(1 row affected)

Total execution time: 00:00:00.047

churn_count,%
307,30.7


**Number & Percentage of Customers who Churned after their Trials**

In [17]:
;WITH ranking AS (
SELECT
customer_id, 
plan_id, 
ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY plan_id) rank
FROM subscriptions 
)
SELECT 
COUNT(*) as 'trial to chrun count',
(100 * COUNT(*) / (SELECT COUNT(DISTINCT customer_id) FROM subscriptions)) as '%'
FROM ranking
WHERE plan_id = 4 AND rank = 2

(1 row affected)

Total execution time: 00:00:00.057

trial to chrun count,%
92,9


**Number & % of Customer Plans after their Trial**

In [18]:
;WITH next_plan AS (
SELECT customer_id, plan_id,
LEAD(plan_id, 1) OVER(PARTITION BY customer_id ORDER BY plan_id)  AS plan_after_trial
FROM subscriptions
)
SELECT plan_after_trial,
COUNT(*) AS conversions, CAST(100 * COUNT(*) / (SELECT COUNT(DISTINCT customer_id)
FROM subscriptions) AS float) AS percentage
FROM next_plan
WHERE plan_after_trial IS NOT NULL
GROUP BY plan_after_trial
ORDER BY plan_after_trial 

(4 rows affected)

Total execution time: 00:00:00.031

plan_after_trial,conversions,percentage
1,546,54
2,539,53
3,258,25
4,307,30


**Number of Customers and Percentage of all Subscriptions at 2020-21-12**

In [8]:
WITH plans_2020 AS (
SELECT 
customer_id, 
s.plan_id, plan_name, 
start_date,
LEAD(start_date, 1) OVER(PARTITION BY customer_id ORDER BY start_date) next_date
FROM subscriptions s JOIN plans p
ON s.plan_id = p.plan_id
WHERE start_date <= '2020-12-31'
),
plans_2021 AS (
SELECT 
plan_id, 
plan_name, 
COUNT(DISTINCT customer_id) AS customers_per_plan
FROM plans_2020
WHERE  next_date IS NOT NULL AND (start_date  < '2020-12-31' AND next_date > '2020-12-31') OR
	(next_date IS NULL AND start_date  < '2020-12-31')
GROUP BY plan_id, plan_name
)
SELECT 
plan_name, 
customers_per_plan, 
CAST(100.0 * customers_per_plan / (SELECT COUNT(DISTINCT customer_id) 
FROM subscriptions) AS float) AS '%'
FROM plans_2021
GROUP BY plan_id, plan_name, customers_per_plan
ORDER BY plan_id

(5 rows affected)

Total execution time: 00:00:00.141

plan_name,customers_per_plan,%
trial,19,1.9
basic monthly,224,22.4
pro monthly,326,32.6
pro annual,195,19.5
churn,235,23.5


**Annual Plan Upgrades 2020**

In [9]:
SELECT COUNT(*) annual_plan_count
FROM subscriptions
WHERE plan_id = 3 AND YEAR(start_date) = '2020'

(1 row affected)

Total execution time: 00:00:00.009

annual_plan_count
195


**Average Time it Took Customers to Upgrade to an Annual Plan**

In [11]:
WITH trial_plans AS (
SELECT customer_id, start_date AS trial_date
FROM subscriptions
WHERE plan_id = 0
),
annual_plans AS (
SELECT customer_id, start_date AS annual_date
FROM subscriptions
WHERE plan_id = 3
)
SELECT AVG(DATEDIFF(DAY, trial_date, annual_date)) AS 'avg days until upgraded to annual plan'
FROM trial_plans tp JOIN annual_plans ap
ON tp.customer_id = ap.customer_id

(1 row affected)

Total execution time: 00:00:00.031

avg days until upgraded to annual plan
104


<span style="color: var(--vscode-foreground);"><b>Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)</b></span>

In [12]:
WITH  trial_plans AS (
SELECT 
customer_id, 
start_date AS trial_date
FROM subscriptions
WHERE plan_id = 0
),
annual_plan AS (
SELECT 
customer_id, 
start_date AS annual_date
FROM subscriptions
WHERE plan_id = 3
),
buckets AS (
SELECT 
tp.customer_id, 
trial_date, 
annual_date, 
DATEDIFF(DAY, trial_date, annual_date) / 30 + 1 AS bucket
FROM trial_plans tp JOIN annual_plan ap
ON tp.customer_id = ap.customer_id
)
SELECT 
CASE
	WHEN bucket = 1 THEN CONCAT(bucket - 1, '-', bucket * 30, ' days')
	ELSE CONCAT((bucket - 1) * 30 + 1, '-', bucket * 30, ' days')
END AS period,
COUNT(customer_id) AS total_customers
FROM buckets
GROUP BY bucket

(12 rows affected)

Total execution time: 00:00:00.069

period,total_customers
0-30 days,48
31-60 days,25
61-90 days,33
91-120 days,35
121-150 days,43
151-180 days,35
181-210 days,27
211-240 days,4
241-270 days,5
271-300 days,1


**Customers who downgraded from a pro-monthly to a basic monthly plan in 2020**

In [13]:
WITH next_plan_cte AS (
SELECT customer_id, plan_id, start_date, 
LEAD(plan_id, 1) OVER(PARTITION BY customer_id ORDER BY plan_id) AS next_plan
FROM subscriptions
)
SELECT COUNT(*) AS downgraded
FROM next_plan_cte
WHERE YEAR(start_date) = '2020' AND plan_id = 2 AND next_plan = 1

(1 row affected)

Total execution time: 00:00:00.184

downgraded
0


**Payments Table**

In [ ]:
create table payments_2020 (
customer_id int,
plan_id int,
plan_name varchar(13),
payment_date date,
amount decimal (5,2),
payment_order int
)



;with cte as (
select 
customer_id,
a.plan_id,
plan_name,
start_date as payment_date,
start_date,
lead(start_date, 1) over(partition by customer_id order by start_date, a.plan_id) as next_date, 
-- use the lead function to show a customer's latest subscription represented by null's, the cutoff date basically.
price as amount
from subscriptions a left join plans b
-- I'm mostly working with the subcriptions table because the only table I need from plans is plan_name.
on a.plan_id = b.plan_id
),
cte_2 as (
select
customer_id,
plan_id,
plan_name,
payment_date,
start_date,
case
	when next_date is null or year(next_date) > '2020' then '2020' 
	else next_date
end as next_date,
-- only show the the dates from next_date columns from 2020.
amount
from cte
where plan_id not in (0, 4)
-- filter out trial plans and churns.
),
cte_3 as (
select
customer_id,
plan_id,
plan_name,
payment_date,
start_date,
next_date,
dateadd(month, -1, next_date) as next_date_2,
-- show the previous month from the next_date column by using the dateadd function to subtract 1 month from the next_date.
amount
from cte_2
),
recursive as (
select 
customer_id,
plan_id,
plan_name,
start_date,
payment_date = (select top 1 start_date from cte_3 where customer_id = a.customer_id and plan_id = a.plan_id),
-- Generate payment dates for customers and a plan combination based on their start date, next_date, and their last subcription payment.
-- the query achives by making the payment's column = a subquery where the customer_id and plan_id columns from the recursive cte
-- match the customer_id and plan_id columns from the 3rd cte.
next_date,
next_date_2,
amount
from cte_3 a

union all
-- union's all the columns from the 3rd cte with the recursive cte

select
customer_id,
plan_id,
plan_name,
start_date,
dateadd(month, - 1, payment_date) as payment_date,
-- subtracts 1 month from the payment_date to show the previous month from the payment_date column.
next_date,
next_date_2,
amount
from recursive date_cte
where payment_date > next_date_2 and plan_id != 3
-- filter the table where the payment_date is ahead of the next_date_2 column which shows the previous months from the original 
-- next_date column and where their are no pro-annual plans
)
insert into payments_2020 (customer_id, plan_id, plan_name, payment_date, amount, payment_order)
select 
customer_id,
plan_id,
plan_name,
payment_date,
amount,
rank() over(partition by customer_id order by customer_id, plan_id, payment_date) as payment_order
-- use the rank function to show the count the number of payments from customers in order so payment 1, payment 2- etc.
from recursive date_cte
where year(payment_date) = '2020'
order by
customer_id,
plan_id,
payment_date

In [15]:
select *
from payments_2020

(5936 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.117

customer_id,plan_id,plan_name,payment_date,amount,payment_order
1,1,basic monthly,2020-01-08,9.90,1
1,1,basic monthly,2020-02-08,9.90,2
1,1,basic monthly,2020-03-08,9.90,3
1,1,basic monthly,2020-04-08,9.90,4
1,1,basic monthly,2020-05-08,9.90,5
1,1,basic monthly,2020-06-08,9.90,6
1,1,basic monthly,2020-07-08,9.90,7
1,1,basic monthly,2020-08-08,9.90,8
2,3,pro annual,2020-09-27,199.00,1
3,1,basic monthly,2020-01-20,9.90,1
